In [1]:
from pathlib import Path

data_dir = Path("../data")

In [2]:
import geopandas as gpd

hel_gdansk_data_dir = data_dir / "raw" / "shapefile" / "emodnet" / "hel_gdansk_data"

gdf = gpd.read_file(hel_gdansk_data_dir / "coastal_type_20210501_0_80k.shp")

In [3]:
import laspy as lp
import numpy as np
from pyproj import Transformer
import os
from tqdm import tqdm

In [4]:
las_file = lp.read(data_dir / "las/N-33-47-C-c-3-4.las")

dataset = np.vstack([las_file.x, las_file.y, las_file.z]).T

In [5]:
def transform_to_lonlat(dataset: np.array) -> np.array:
    """
    Convert coordinates from .las file to longitude and latitude.

    Args:
        dataset (np.array): Array of coordinates.

    Returns:
        np.array: Array of longitude and latitude coordinates.
    """

    source = "EPSG:2180"  # PL-1992 is the projection used in the dataset
    dest = "EPSG:4326"  # WGS84, the standard for GPS coordinates used all over the world

    # Create a transformer object
    transformer = Transformer.from_crs(source, dest, always_xy=True)

    x = dataset[:, 0]
    y = dataset[:, 1]
    z = dataset[:, 2]

    # Transform the coordinates
    lon, lat = transformer.transform(x, y)
    # Create a new array with the transformed coordinates
    transformed_coordinates = np.vstack((lon, lat, z)).T
    return transformed_coordinates

In [6]:
transformed_dataset = transform_to_lonlat(dataset)

In [7]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# lidar_data is a NumPy array with shape (n, 3)
# Columns: x, y, z
lidar_df = pd.DataFrame(transformed_dataset, columns=["x", "y", "z"])
lidar_df["geometry"] = [Point(xy) for xy in zip(lidar_df["x"], lidar_df["y"])]
lidar_gdf = gpd.GeoDataFrame(lidar_df, geometry="geometry", crs=gdf.crs)  # Make sure CRS matches!

In [8]:
lidar_gdf = lidar_gdf.to_crs(gdf.crs)

In [9]:
merged_gdf = gpd.sjoin_nearest(
    lidar_gdf,
    gdf[["coasttype", "geometry"]],  # Only need coasttype and geometry
    how="left",
    distance_col="dist_to_line",
)

O:\studia\DataScienceMasters\10th_semester\lidar_classification\.venv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


In [10]:
final_df = merged_gdf[["x", "y", "z", "coasttype"]]
final_df

,x,y,z,coasttype
0,17.047468,54.670739,-2.79,Sand beach fronting upland (> 1 Km long)
1,17.047475,54.670739,-2.78,Sand beach fronting upland (> 1 Km long)
2,17.047483,54.670739,-2.77,Sand beach fronting upland (> 1 Km long)
3,17.047491,54.670739,-2.76,Sand beach fronting upland (> 1 Km long)
4,17.047429,54.670734,-2.80,Sand beach fronting upland (> 1 Km long)
...,...,...,...,...
2687846,17.062461,54.666668,1.16,Sand beach fronting upland (> 1 Km long)
2687847,17.062469,54.666668,1.14,Sand beach fronting upland (> 1 Km long)
2687848,17.062477,54.666669,1.09,Sand beach fronting upland (> 1 Km long)
2687849,17.062485,54.666669,1.04,Sand beach fronting upland (> 1 Km long)


In [14]:
final_df = pd.DataFrame(columns=["x", "y", "z", "coasttype"])
i = 0

for file in tqdm(os.listdir("../data/las")):
    if file.endswith(".las") and "N-34-37" in file:
        las_file = lp.read(os.path.join("../data/las", file))
        dataset = np.vstack([las_file.x, las_file.y, las_file.z]).T
        transformed_dataset = transform_to_lonlat(dataset)
        lidar_df = pd.DataFrame(transformed_dataset, columns=["x", "y", "z"])
        lidar_df["geometry"] = [Point(xy) for xy in zip(lidar_df["x"], lidar_df["y"])]
        lidar_gdf = gpd.GeoDataFrame(lidar_df, geometry="geometry", crs=gdf.crs)
        lidar_gdf = lidar_gdf.to_crs(gdf.crs)
        merged_gdf = gpd.sjoin_nearest(
            lidar_gdf,
            gdf[["coasttype", "geometry"]],  # Only need coasttype and geometry
            how="left",
            distance_col="dist_to_line",
        )
        final_df = pd.concat([final_df, merged_gdf[["x", "y", "z", "coasttype"]]])

        i += 1
        if i == 10:
            break

  0%|          | 0/294 [00:00<?, ?it/s]O:\studia\DataScienceMasters\10th_semester\lidar_classification\.venv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(
C:\Users\mikol\AppData\Local\Temp\ipykernel_27212\1796992843.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, merged_gdf[["x", "y", "z", "coasttype"]]])
 24%|██▍       | 71/294 [00:26<01:21,  2.73it/s]O:\studia\DataScienceMasters\10th_semester\lidar_classification\.venv\Lib\site-packages\geopandas\array.py:403: UserWarning: Geometry is in a geographic CR

In [15]:
final_df.coasttype.value_counts()

coasttype
Sand beach fronting upland (> 1 Km long)    12514137
Name: count, dtype: int64